In [ ]:
import pandas as pd
from keras.models import Sequential
from keras.layers import LSTM, Dense, Dropout
import numpy as np
from sklearn.preprocessing import StandardScaler
import matplotlib.pyplot as plt
from typing import Tuple

In [ ]:
# Constants
SEQUENCE_LEN = 21 * 4              # 21 days of 6-hours data
OUTPUT_LEN = 5 * 4                 # 5 days of 6-hours data
INPUT_FEATURES = 23                # Number of input features (e.g., temperature, humidity, etc.)
OUTPUT_FEATURES = 5 * OUTPUT_LEN   # Predicting flatten 4 features for each timestamp (wind direction is described as two cols)

In [ ]:
def wind_direction_sin_cos(data: pd.DataFrame):
    """ To avoid problem of huge MSE error degrees are replaced by sin and cos functions """
    data['Kierunek wiatru [sin]'] = np.sin(data['Kierunek wiatru  [°]'] * 2 * np.pi / 360)  # transform to radians and calculate sin
    data['Kierunek wiatru [cos]'] = np.cos(data['Kierunek wiatru  [°]'] * 2 * np.pi / 360)  # transform to radians and calculate sin
    data.drop('Kierunek wiatru  [°]', axis=1, inplace=True)
    return data

def create_sequences(data: pd.DataFrame):
    X, y = [], []
    for i in range(len(data) - SEQUENCE_LEN - OUTPUT_LEN):
        X.append(data[i: i + SEQUENCE_LEN])
        y.append(data[i + SEQUENCE_LEN: i + SEQUENCE_LEN + OUTPUT_LEN, :5].flatten()) # Taking the first 4 features for the next 24 hours
    return np.array(X), np.array(y)

def invert_scale_for_X(X: np.ndarray, scaler):
    copy_X = X.copy()
    for i in range(X.shape[-1]):
        print()
        std, mean = scaler.scale_[i], scaler.mean_[i]
        copy_X[:, :, i] = X[:, :, i] * std + mean
    return copy_X

def invert_scale_for_y(y: np.ndarray, scaler):
    copy_y = y.copy()
    for i in range(5):
        std, mean = scaler.scale_[i], scaler.mean_[i]
        copy_y[:, i::5] = y[:, i::5] * std + mean
    return copy_y

def back_to_degrees_for_X(X: np.ndarray):
    sin_part, cos_part = X[:, :, 2], X[:, :, 3]  # get all 
    X[:, :, 2] = np.arctan2(sin_part, cos_part) * 180 / np.pi
    replaced_X = np.delete(X, 3, axis=2)
    return replaced_X

def back_to_degrees_for_y(y: np.ndarray):
    sin_part, cos_part = y[:, 2::5], y[:, 3::5]  # get all 
    y[:, 2::5] = np.arctan2(sin_part, cos_part) * 180 / np.pi
    replaced_y = np.delete(y, list(range(3, 99, 5)), axis=1)
    return replaced_y

def plot_one_prediction(idx: int, X_true, y_true, y_pred, figsize: Tuple[int] = (14, 9)):
    x = range(-SEQUENCE_LEN, OUTPUT_LEN)
    x1, x2 = x[1:1 + SEQUENCE_LEN], x[SEQUENCE_LEN:]
    labels = ['Temperatura [°C]', 'Ciśnienie atmos. [hPa]', 'Kierunek wiatru [°]', 'Prędkość wiatru [m/s]']

    fig, axs = plt.subplots(4, figsize=figsize)
    fig.suptitle('Porównanie wyników modelu z faktycznym stanem pogody')
    for i in range(4):
        axs[i].plot(x1, X_true[idx + 1, :, i], label='Dane wejściowe', color='green')
        axs[i].plot(x2, y_true[:, i::4][idx], label='Wartość prawdziwa', color='blue')
        axs[i].plot(x2, y_pred[:, i::4][idx], label='Predykcja', color='red')
        axs[i].set_ylabel(labels[i])
        axs[i].grid()
        axs[i].legend()
    axs[3].set_xlabel('Czas [godz.]')
    plt.show()
    
def BIAS(y_true: np.ndarray, y_pred: np.ndarray):
    T_bias = np.mean(y_pred[:, 0::4] - y_true[:, 0::4], axis=0).reshape(1, -1)    # bias for temperature
    P_bias = np.mean(y_pred[:, 1::4] - y_true[:, 1::4], axis=0).reshape(1, -1)    # bias for pressure
    
    WD_diff = np.mod(y_pred[:, 2::4], 360) - np.mod(y_true[:, 2::4], 360)
    WD_diff = np.where(np.abs(WD_diff) > 180, WD_diff - np.sign(WD_diff) * 360, WD_diff)
    WD_bias = np.mean(WD_diff, axis=0).reshape(1, -1)  # bias for wind direction
    
    WV_bias = np.mean(np.abs(y_pred[:, 3::4] - y_true[:, 3::4]), axis=0).reshape(1, -1)   # bias for wind velocity
    
    BIAS_by_hour = np.append(np.append(np.append(T_bias, P_bias, axis=0), WD_bias, axis=0), WV_bias, axis=0)
    return BIAS_by_hour

def MAE(y_true: np.ndarray, y_pred: np.ndarray):
    T_mae = np.mean(np.abs(y_pred[:, 0::4] - y_true[:, 0::4]), axis=0).reshape(1, -1)    # mae for temperature
    P_mae = np.mean(np.abs(y_pred[:, 1::4] - y_true[:, 1::4]), axis=0).reshape(1, -1)    # mae for pressure
    
    WD_diff = np.mod(y_pred[:, 2::4], 360) - np.mod(y_true[:, 2::4], 360)
    WD_diff = np.where(np.abs(WD_diff) > 180, WD_diff - np.sign(WD_diff) * 360, WD_diff)
    WD_mae = np.mean(np.abs(WD_diff), axis=0).reshape(1, -1)  # mae for wind direction
    
    WV_mae = np.mean(np.abs(y_pred[:, 3::4] - y_true[:, 3::4]), axis=0).reshape(1, -1)   # mae for wind velocity
    
    MAE_by_hour = np.append(np.append(np.append(T_mae, P_mae, axis=0), WD_mae, axis=0), WV_mae, axis=0)
    return MAE_by_hour
    
    
def RMSE(y_true: np.ndarray, y_pred: np.ndarray):
    T_mse = np.mean((y_true[:, 0::4] - y_pred[:, 0::4])**2, axis=0).reshape(1, -1)
    P_mse = np.mean((y_true[:, 1::4] - y_pred[:, 1::4])**2, axis=0).reshape(1, -1)
    WD_diff = np.mod(y_pred[:, 2::4], 360) - np.mod(y_true[:, 2::4], 360)
    WD_diff = np.where(np.abs(WD_diff) > 180, WD_diff - np.sign(WD_diff) * 360, WD_diff)
    WD_mse = np.mean(WD_diff**2, axis=0).reshape(1, -1)  # bias for wind direction
    WV_mse = np.mean((y_true[:, 3::4] - y_pred[:, 3::4])**2, axis=0).reshape(1, -1)
    MSE_by_hour = np.append(np.append(np.append(T_mse, P_mse, axis=0), WD_mse, axis=0), WV_mse, axis=0)
    
    return np.sqrt(MSE_by_hour)

In [ ]:
data = pd.read_csv('../data/preprocessed_data/complete_krk_2017-22.csv')
data['timestamp'] = pd.to_datetime(data['timestamp'])
data = data.set_index('timestamp')
data_6h = data[data.index.hour.isin([0, 6, 12, 18])]
data_6h = wind_direction_sin_cos(data_6h)
data_6h

In [ ]:
cols = data_6h.columns
new_order = [0, 1, 21, 22, 11] + list(range(2, 11)) + list(range(12, 21))
cols = [cols[i] for i in new_order]
data_6h = data_6h[cols]
data_6h

In [ ]:
# Assuming 'data' is your dataset with shape (total_hours, 26_features)
# and 'total_hours' is a multiple of 24

# Normalize your data
scaler = StandardScaler()
data_normalized = scaler.fit_transform(data_6h)
data_normalized

In [ ]:
# Create sequences
X, y = create_sequences(data_normalized)

In [ ]:
train_size = 4 * (365 * 3 + 366)  # train data includes first 4 years (2017-20)
val_size = 4 * 365                # validation data includes the next 2021 year  
test_size = 4 * 365               # test data includes the last 2022 year

X_train, y_train = X[:train_size], y[:train_size]
X_val, y_val = X[train_size:train_size+val_size], y[train_size:train_size+val_size]
X_test, y_test = X[train_size+val_size:], y[train_size+val_size:]

In [ ]:
# Define the model
model = Sequential()

# LSTM layers
model.add(LSTM(256, return_sequences=True, input_shape=(SEQUENCE_LEN, INPUT_FEATURES)))
model.add(LSTM(128, return_sequences=False))
# Dense layer
model.add(Dense(128, activation='ReLU'))
# Dropout layer to prevent overfitting
model.add(Dropout(0.2))
# Output layer
model.add(Dense(OUTPUT_FEATURES, activation='linear'))  # 'linear' activation for regression tasks

model.compile(optimizer='adam', loss='mean_squared_error')
model.summary()

In [ ]:
history = model.fit(X_train, y_train, epochs=20, batch_size=32, validation_data=(X_val, y_val))

In [ ]:
model.save('LSTM_5d_08-01-2024_final')

In [ ]:

from keras.models import load_model

model = load_model('LSTM_5d_08-01-2024_final')


In [35]:
january_t = (0, 31*4)
april_t = ((31+28+31)*4, (31+28+31+30)*4)
july_t = ((31+28+31+30+31+30)*4, (31+28+31+30+31+30+31)*4)
october_t = ((31+28+31+30+31+30+31+30+31)*4, (31+28+31+30+31+30+31+30+31+30)*4)

In [48]:
# Make predictions
y_pred = model.predict(X_test[october_t[0]:october_t[1]])

4/4 [==============================] - 1s 111ms/step


In [49]:
inv_y_pred = invert_scale_for_y(y_pred, scaler)
real_y_pred = back_to_degrees_for_y(inv_y_pred)

inv_y_test = invert_scale_for_y(y_test, scaler)
real_y_test = back_to_degrees_for_y(inv_y_test)

inv_X_test = invert_scale_for_X(X_test, scaler)
real_X_test = back_to_degrees_for_X(inv_X_test)

In [ ]:
plot_one_prediction(900, real_X_test, real_y_test, real_y_pred)

In [53]:
bias_5d = BIAS(real_y_test[october_t[0]:october_t[1]], real_y_pred)
np.min(bias_5d, axis=1)

array([  0.73549634,  -2.560413  , -33.04201369,   1.65151883])

In [56]:
mae_5d = MAE(real_y_test[october_t[0]:october_t[1]], real_y_pred)
(np.average(mae_5d, axis=1))

array([ 3.15784797,  7.77155419, 79.01964965,  1.85895146])

In [59]:
rmse_5d = RMSE(real_y_test[october_t[0]:october_t[1]], real_y_pred)
np.average(rmse_5d, axis=1)

array([ 3.92572624,  8.9628261 , 96.35331641,  2.2457825 ])

In [ ]:
x = range(0, (OUTPUT_LEN+1)*3, 6)
fig, axs = plt.subplots(3, figsize=(6, 7))
fig.suptitle('Temperatura', fontsize='xx-large')
axs[0].stem(x[1:], bias_5d[0, :10])
axs[0].set_ylabel('BIAS [°C]')
axs[0].grid()
axs[0].set_xticks(x)
axs[1].stem(x[1:], mae_5d[0, :10])
axs[1].set_ylabel('MAE [°C]')
axs[1].grid()
axs[1].set_xticks(x)
axs[2].stem(x[1:], rmse_5d[0, :10])
axs[2].set_ylabel('RMSE [°C]')
axs[2].grid()
axs[2].set_xlabel('Czas [godz.]')
axs[2].set_xticks(x)
plt.show()

In [ ]:
x = range(0, (OUTPUT_LEN+1)*3, 6)
fig, axs = plt.subplots(3, figsize=(6, 7))
fig.suptitle('Ciśnienie atmosferyczne', fontsize='xx-large')
axs[0].stem(x[1:], bias_5d[1, :10])
axs[0].set_ylabel('BIAS [hPa]')
axs[0].grid()
axs[0].set_xticks(x)
axs[1].stem(x[1:], mae_5d[1, :10])
axs[1].set_ylabel('MAE [hPa]')
axs[1].grid()
axs[1].set_xticks(x)
axs[2].stem(x[1:], rmse_5d[1, :10])
axs[2].set_ylabel('RMSE [hPa]')
axs[2].grid()
axs[2].set_xlabel('Czas [godz.]')
axs[2].set_xticks(x)
plt.show()

In [ ]:
x = range(0, (OUTPUT_LEN+1)*3, 6)
fig, axs = plt.subplots(3, figsize=(6, 7))
fig.suptitle('Kierunek wiatru', fontsize='xx-large')
axs[0].stem(x[1:], bias_5d[2, :10])
axs[0].set_ylabel('BIAS [°]')
axs[0].grid()
axs[0].set_xticks(x)
axs[1].stem(x[1:], mae_5d[2, :10])
axs[1].set_ylabel('MAE [°]')
axs[1].grid()
axs[1].set_xticks(x)
axs[2].stem(x[1:], rmse_5d[2, :10])
axs[2].set_ylabel('RMSE [°]')
axs[2].grid()
axs[2].set_xlabel('Czas [godz.]')
axs[2].set_xticks(x)
plt.show()

In [ ]:
x = range(0, (OUTPUT_LEN+1)*3, 6)
fig, axs = plt.subplots(3, figsize=(6, 7))
fig.suptitle('Prędkość wiatru', fontsize='xx-large')
axs[0].stem(x[1:], bias_5d[3, :10])
axs[0].set_ylabel('BIAS [m/s]')
axs[0].grid()
axs[0].set_xticks(x)
axs[1].stem(x[1:], mae_5d[3, :10])
axs[1].set_ylabel('MAE [m/s]')
axs[1].grid()
axs[1].set_xticks(x)
axs[2].stem(x[1:], rmse_5d[3, :10])
axs[2].set_ylabel('RMSE [m/s]')
axs[2].grid()
axs[2].set_xlabel('Czas [godz.]')
axs[2].set_xticks(x)
plt.show()